# How robo-advisors make investment decisions

The examples below demonstrate the basic concept of how robo-advisors make investment decisions:

- **Basic portfolio optimization using Modern Portfolio Theory** (covered in much greater detail last time)
- Tax-loss harvesting logic
- Risk profiling and asset allocation
- Portfolio rebalancing

We will use these examples to:

- Demonstrate how robo-advisors make investment decisions
- Show how risk profiles are translated into portfolio allocations
- Illustrate tax-loss harvesting logic
- Demonstrate portfolio rebalancing mechanics

Each example can be expanded. You may want to try:

- Add additional (or interactive) visualization using `matplotlib` or `plotly`
- Include more sophisticated optimization techniques (think `riskfolio` package)
- Add error handling and input validation (useful for more advanced and user-friendly applications)
- Include more complex tax rules
- Add additional real-time market data integration (more stocks, different time periods, alternative assets).

## Risk Profiling and Portfolio Optimization Example

In [40]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.optimize import minimize
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import seaborn as sns

In [41]:
class RoboAdvisor:
    def __init__(self):
        self.risk_free_rate = 0.02
        
    def get_historical_data(self, tickers, start_date, end_date):
        data = pd.DataFrame()
        for ticker in tickers:
            stock_data = yf.download(ticker, start=start_date, end=end_date)['Adj Close']
            data[ticker] = stock_data
        return data
    
    def calculate_portfolio_metrics(self, returns):
        mean_returns = returns.mean() * 252
        cov_matrix = returns.cov() * 252
        return mean_returns, cov_matrix
    
    def optimize_portfolio(self, returns, risk_score):
        mean_returns, cov_matrix = self.calculate_portfolio_metrics(returns)
        num_assets = len(returns.columns)
        
        target_vol = (risk_score / 10) * 0.20 + 0.05
        
        def objective(weights):
            portfolio_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
            return (portfolio_vol - target_vol)**2
        
        constraints = [
            {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
        ]
        
        bounds = tuple((0, 1) for asset in range(num_assets))
        initial_weights = np.array([1/num_assets] * num_assets)
        
        result = minimize(objective, initial_weights, method='SLSQP',
                        bounds=bounds, constraints=constraints)
        
        return result.x
    
    def calculate_portfolio_performance(self, weights, returns):
        mean_returns, cov_matrix = self.calculate_portfolio_metrics(returns)
        
        portfolio_return = np.sum(mean_returns * weights)
        portfolio_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        sharpe_ratio = (portfolio_return - self.risk_free_rate) / portfolio_vol
        
        return {
            'return': portfolio_return,
            'volatility': portfolio_vol,
            'sharpe_ratio': sharpe_ratio
        }

def plot_results(weights, tickers, metrics):
    plt.figure(figsize=(15, 5))
    
    # Portfolio allocation pie chart
    plt.subplot(1, 2, 1)
    colors = sns.color_palette("husl", len(tickers))
    plt.pie(weights, labels=tickers, autopct='%1.1f%%', colors=colors)
    plt.title('Optimal Portfolio Allocation')
    
    # Portfolio metrics bar chart
    plt.subplot(1, 2, 2)
    metrics_data = [metrics['return'], metrics['volatility'], metrics['sharpe_ratio']]
    bars = plt.bar(['Expected\nReturn', 'Volatility', 'Sharpe\nRatio'], metrics_data)
    
    # Color code the bars
    bars[0].set_color('green')
    bars[1].set_color('red')
    bars[2].set_color('blue')
    
    plt.title('Portfolio Metrics')
    plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    
    # Format y-axis for percentage display on return and volatility
    def format_value(x, p):
        if abs(x) >= 1:
            return f'{x:.1f}'
        else:
            return f'{x:.1%}'
    
    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(format_value))
    
    plt.tight_layout()
    plt.show()

def update_portfolio(change):
    """
    Update portfolio based on risk score
    """
    risk_score = change['new'] if isinstance(change, dict) else change
    
    # Get optimal weights
    optimal_weights = robo.optimize_portfolio(returns, risk_score)
    
    # Calculate portfolio metrics
    metrics = robo.calculate_portfolio_performance(optimal_weights, returns)
    
    # Clear previous output and display results
    clear_output(wait=True)
    
    # Display the slider first
    display(risk_slider)
    
    # Display portfolio information
    print(f"\nRisk Score: {risk_score}")
    print("\nOptimal Portfolio Allocation:")
    for ticker, weight in zip(tickers, optimal_weights):
        print(f"{ticker}: {weight:.2%}")
        
    print(f"\nPortfolio Metrics:")
    print(f"Expected Annual Return: {metrics['return']:.2%}")
    print(f"Expected Annual Volatility: {metrics['volatility']:.2%}")
    print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
    
    # Plot results
    plot_results(optimal_weights, tickers, metrics)


In [42]:
# Initialize robo advisor and get data
robo = RoboAdvisor()
tickers = ['SPY', 'AGG', 'VEA', 'VWO']
data = robo.get_historical_data(tickers, '2018-01-01', '2023-12-31')
returns = data.pct_change().dropna()


# Create the slider widget
risk_slider = widgets.IntSlider(
    value=5,
    min=1,
    max=10,
    step=1,
    description='Risk Score:',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# Observe slider changes
risk_slider.observe(update_portfolio, names='value')

# Display the slider (initial display)
display(risk_slider)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


IntSlider(value=5, continuous_update=False, description='Risk Score:', layout=Layout(width='50%'), max=10, min…

Change the acceptable risk score value through the interactive slider and observe the results in the log window of your Python instance (see the bottom of the screen for the log entries icon):
![image](https://raw.githubusercontent.com/VitaliAlexeev/AI_Investments_2024/refs/heads/main/figs/Lec04_example01.png)
